# RIPEMD-256 Implementation

A pure Python implementation of the RIPEMD-256 cryptographic hash function.

## Overview

RIPEMD-256 is a 256-bit cryptographic hash function that extends the RIPEMD-128 algorithm. It uses two parallel computation lines and produces a 256-bit hash output, offering enhanced security through its larger digest size while maintaining the structural design principles of the RIPEMD family.

## Features

- **Pure Python Implementation**: No external dependencies required
- **256-bit Security**: Provides stronger security than RIPEMD-128
- **Dual Line Processing**: Uses two parallel computation lines like other RIPEMD variants
- **Flexible Input**: Accepts both string and bytes input
- **Standard Compliance**: Implements the RIPEMD-256 specification

## Functions

### `RIPEMD_256_hash(message_str)`
Main hashing function that:
- Accepts string or bytes input
- Processes message in 512-bit blocks
- Uses two parallel computation lines with state exchanges
- Returns 256-bit hash as bytes object

### Core Algorithm Components

#### `preprocess(message_bytes)`
Message padding function:
- Appends 0x80 padding bit
- Pads with zeros to achieve proper block alignment
- Appends original message bit length

#### Round Functions
- `f(j, x, y, z)`: Four different boolean functions based on round number
- `K(j)`, `k_dash(j)`: Round constants for left and right computation lines
- `r(j)`, `r_dash(j)`: Message word selection patterns
- `s(j)`, `s_dash(j)`: Bit rotation amounts for each round

#### `rol(x, s)`
32-bit left rotation function for bit manipulation operations.

## Algorithm Specifications

RIPEMD-256 characteristics:
- **Block size**: 512 bits (64 bytes)
- **Output size**: 256 bits (32 bytes)
- **Rounds**: 64 rounds per line (2 parallel lines)
- **Initial values**: Eight 32-bit constants (h0-h7)
- **State exchanges**: Variables exchanged between lines at specific rounds
- **Final combination**: Direct addition of computed values to initial hash state

## Key Algorithm Features

### Dual Line Structure
RIPEMD-256 uses two parallel computation lines:
- **Left line**: Processes with standard round functions
- **Right line**: Uses inverted round function sequence
- **State exchanges**: Variables swap between lines at rounds 15, 31, 47, 63

### Round Function Progression
Each line uses four different boolean functions:
1. **Rounds 0-15**: XOR operation (`x ^ y ^ z`)
2. **Rounds 16-31**: Conditional selection (`(x & y) | (~x & z)`)
3. **Rounds 32-47**: Mixed OR/XOR (`(x | ~y) ^ z`)
4. **Rounds 48-63**: Alternative selection (`(x & z) | (y & ~z)`)

## Example Outputs

```python
# Test with different inputs
print(RIPEMD_256_hash("Hello, World!").hex())
# Output: 567750c6d34dcba7ae038a80016f3ca3260ec25bfdb0b68bbb8e730b00b2447d

print(RIPEMD_256_hash("The quick brown fox jumps over the lazy dog").hex())
# Output: c3b0c2f764ac6d576a6c430fb61a6f2255b4fa833e094b1ba8c1e29b6353036f

print(RIPEMD_256_hash("this is a pure implementation of RIPEMD - 256 ").hex())
# Output: b77a593de38fb2f2da7f9cda58ee5842f13b78c7a573a930f6e9218c44378cb3
```

## Security Considerations

RIPEMD-256 provides:
- **Enhanced security**: 256-bit output provides stronger collision resistance
- **Dual line design**: Parallel processing increases complexity for attacks  
- **Variable exchanges**: State swapping between lines adds cryptographic strength
- **Proven structure**: Based on well-analyzed RIPEMD design principles

## Technical Details

### Initial Hash Values
```python
h0 = 0x67452301    h1 = 0xEFCDAB89    h2 = 0x98BADCFE    h3 = 0x10325476
h4 = 0x76543210    h5 = 0xFEDCBA98    h6 = 0x89ABCDEF    h7 = 0x01234567
```

### State Variable Management
The algorithm maintains 8 state variables (4 per line) and performs controlled exchanges during processing to ensure both lines contribute to the final hash value.

### Message Processing
- Input is padded to multiple of 512 bits
- Each 512-bit block is processed through 64 rounds
- Two computation lines work in parallel with periodic state exchanges
- Final hash is combination of all 8 state variables

## Applications

RIPEMD-256 is suitable for:
- Applications requiring 256-bit hash security
- Systems needing RIPEMD family compatibility
- Cryptographic protocols requiring medium-strength hashing
- Research and educational purposes

## Compatibility

- **Python 3.6+**: Compatible with modern Python versions
- **Cross-platform**: Works on all Python-supported systems
- **Memory efficient**: Processes data in fixed-size blocks
- **Standard compliant**: Follows RIPEMD-256 specification

In [1]:
def preprocess(message_bytes):
    if(not isinstance(message_bytes, bytes)):
        raise TypeError("Input must be of type bytes")
    m = bytearray(message_bytes)
    bit_length = len(m) * 8

    # Append 0x80 (10000000 in binary)
    m.append(0x80)

    # Compute number of zero bytes to append
    padding_len = (56 - (len(m) % 64)) % 64
    m.extend([0x00] * padding_len)

    # Append original bit length as a 64-bit little-endian integer
    m.extend(bit_length.to_bytes(8, byteorder='little'))

    return m

In [2]:
def f(j,x,y,z):
    if(j>=0 and j<=15):
        return x^ y ^ z
    elif(j>=16 and j<=31):
        return (x & y) | (~x & z)   
    elif(j>=32 and j<=47):
        return (x | ~y) ^ z
    elif(j>=48 and j<=63):
        return (x & z) | (y & ~z)

In [4]:
def K(j):
    if(j>=0 and j<=15):
        return 0x00000000
    elif(j>=16 and j<=31):
        return 0x5A827999
    elif(j>=32 and j<=47):
        return 0x6ED9EBA1
    elif(j>=48 and j<=63):
        return 0x8F1BBCDC
def k_dash(j):
    if(j>=0 and j<=15):
        return 0x50A28BE6
    elif(j>=16 and j<=31):
        return 0x5C4DD124
    elif(j>=32 and j<=47):
        return 0x6D703EF3
    elif(j>=48 and j<=63):
        return 0x00000000

In [5]:
def r(j):
    if(j>=0 and j<=15):
        return j
    elif(j>=16 and j<=31):
        return [7, 4, 13, 1, 10, 6, 15, 3, 12, 0, 9, 5, 2, 14, 11, 8][j-16]
    elif(j>=32 and j<=47):
        return [3, 10, 14, 4, 9, 15, 8, 1, 2, 7, 0, 6, 13, 11, 5, 12][j-32]
    elif(j>=48 and j<=63):
        return [1, 9, 11, 10, 0, 8, 12, 4, 13, 3, 7, 15, 14, 5, 6, 2][j-48]
def r_dash(j):
    if(j>=0 and j<=15):
        return [5, 14, 7, 0, 9, 2, 11, 4, 13, 6, 15, 8, 1, 10, 3, 12][j]
    elif(j>=16 and j<=31):
        return [6, 11, 3, 7, 0, 13, 5, 10, 14, 15, 8, 12, 4, 9, 1, 2][j-16]
    elif(j>=32 and j<=47):
        return [15, 5, 1, 3, 7, 14, 6, 9, 11, 8, 12, 2, 10, 0, 4, 13][j-32]
    elif(j>=48 and j<=63):
        return [8, 6, 4, 1, 3, 11, 15, 0, 5, 12, 2, 13, 9, 7, 10, 14][j-48]

In [6]:
def s(j):
    if(j>=0 and j<=15):
        return [11, 14, 15, 12, 5, 8, 7, 9, 11, 13, 14, 15, 6, 7, 9, 8][j]
    elif(j>=16 and j<=31):
        return [7, 6, 8, 13, 11, 9, 7, 15, 7, 12, 15, 9, 11, 7, 13, 12][j-16]
    elif(j>=32 and j<=47):
        return [11, 13, 6, 7, 14, 9, 13, 15, 14, 8, 13, 6, 5, 12, 7, 5][j-32]
    elif(j>=48 and j<=63):
        return [11,12 ,14 ,15 ,14 ,15 ,9 ,8 ,9 ,14 ,5 ,6 ,8 ,6 ,5 ,12][j-48]
def s_dash(j):
    if(j>=0 and j<=15):
        return [8, 9, 9, 11, 13, 15, 15, 5, 7, 7, 8, 11, 14, 14, 12, 6][j]
    elif(j>=16 and j<=31):
        return [9, 13, 15, 7, 12, 8, 9, 11, 7, 7, 12, 7, 6, 15, 13, 11][j-16]
    elif(j>=32 and j<=47):
        return [9, 7, 15, 11, 8, 6, 6, 14, 12, 13, 5, 14, 13, 13, 7, 5][j-32]
    elif(j>=48 and j<=63):
        return [15 ,5 ,8 ,11 ,14 ,14 ,6 ,14 ,6 ,9 ,12 ,9 ,12 ,5 ,15 ,8][j-48]

In [8]:
h0 = 0x67452301; h1 = 0xEFCDAB89; h2 = 0x98BADCFE; h3 = 0x10325476;
h4 =0x76543210; h5 = 0xFEDCBA98; h6 = 0x89ABCDEF; h7 = 0x01234567;

In [9]:
"""
rols denotes cyclic left shift (rotate) over s bit positions.
"""
def rol(x, s):
    return ((x << s) | (x >> (32 - s))) & 0xFFFFFFFF

In [15]:
def RIPEMD_256_hash(message_str):
    # Initial hash values
    h0= 0x67452301
    h1= 0xEFCDAB89
    h2= 0x98BADCFE
    h3= 0x10325476
    h4= 0x76543210
    h5= 0xFEDCBA98
    h6= 0x89ABCDEF
    h7= 0x01234567
    if isinstance(message_str, str):
        message = message_str.encode("utf-8")
    else:
        message = message_str
    padded_message = preprocess(message)
    
    # Process the message in successive 512-bit chunks
    for i in range(0, len(padded_message), 64):
        block = padded_message[i:i+64]
        
        # Break chunk into sixteen 32-bit little-endian words
        X = []
        for j in range(16):
            X.append(int.from_bytes(block[j*4:j*4+4], byteorder='little'))
        
        # Initialize hash value for this chunk
        A = h0; B = h1; C = h2; D = h3;
        A_prime = h4; B_prime = h5; C_prime = h6; D_prime = h7; 
        
        # Main loop
        for j in range(64):
            # Left line
            T = (A + f(j, B, C, D) + X[r(j)] + K(j)) & 0xFFFFFFFF
            T = rol(T, s(j))
            A = D
            D = C
            C = B
            B = T
            # Right line
            T_prime = (A_prime + f(63-j, B_prime, C_prime, D_prime) + X[r_dash(j)] + k_dash(j)) & 0xFFFFFFFF
            T_prime = rol(T_prime, s_dash(j))
            A_prime = D_prime
            D_prime = C_prime
            C_prime = B_prime
            B_prime = T_prime
            if(j==15):
                T = A ; A= A_prime ; A_prime = T;
            elif(j==31):
                T = B ; B = B_prime ; B_prime = T;
            elif(j==47):
                T = C ; C= C_prime ; C_prime = T;
            elif(j==63):
                T = D ; D = D_prime ; D_prime = T; 
        # Combine results
        h0=(h0+A) & 0xFFFFFFFF;
        h1=(h1+B) & 0xFFFFFFFF;
        h2=(h2+C) & 0xFFFFFFFF;
        h3=(h3+D)   & 0xFFFFFFFF;
        h4=(h4+A_prime) & 0xFFFFFFFF;
        h5=(h5+B_prime) & 0xFFFFFFFF;
        h6=(h6+C_prime) & 0xFFFFFFFF;
        h7=(h7+D_prime)   & 0xFFFFFFFF;
    
    return (h0.to_bytes(4, byteorder='little') + 
            h1.to_bytes(4, byteorder='little') + 
            h2.to_bytes(4, byteorder='little') + 
            h3.to_bytes(4, byteorder='little') + 
            h4.to_bytes(4, byteorder='little')+
            h5.to_bytes(4, byteorder='little') +
            h6.to_bytes(4, byteorder='little') +    
            h7.to_bytes(4, byteorder='little')
            )

In [17]:
print(RIPEMD_256_hash("Hello, World!").hex())

567750c6d34dcba7ae038a80016f3ca3260ec25bfdb0b68bbb8e730b00b2447d


In [28]:
print(RIPEMD_256_hash("The quick brown fox jumps over the lazy dog").hex())
print(RIPEMD_256_hash("this is a pure implementation of RIPEMD - 256 ").hex())

c3b0c2f764ac6d576a6c430fb61a6f2255b4fa833e094b1ba8c1e29b6353036f
b77a593de38fb2f2da7f9cda58ee5842f13b78c7a573a930f6e9218c44378cb3
